In [2]:
import import_ipynb
from la_lib import fn_trainingdata_create 
from la_lib import fn_randomforest_modle_create 
from la_lib import fn_predictingdata_create 
from la_lib import dbconn 

# RandomForest활용하여 모델 만들기
from sklearn import svm, metrics
from sklearn.ensemble import RandomForestClassifier
import random, re
import numpy 
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os
import datetime as dt
import pymssql  
import pandas as pd
import configparser
 
config = configparser.ConfigParser()
config.read('../include/config.ini')

host = config['DB']['SERVER']
username = config['DB']['USER']
pwd = config['DB']['PASSWORD']
db = config['DB']['database']

year = config['info']['year']
term = config['info']['term']
version =config['info']['version']
conn = dbconn()


In [3]:
cur = conn.cursor()


In [15]:
query2 ="select (select category_nm from sjcuwebent.dbo.jwcategory where category_id = a.category_id )as cate_nm ,article_title, article_text  "
query2 = query2  + " from sjcuwebent.[dbo].[jwarticle] a  where  board_no = '41' and delete_yn = 'N' order by category_id "

print(query2)

df_SjcuWorkingHistory = pd.read_sql(query2, conn)

df_SjcuWorkingHistory

select (select category_nm from sjcuwebent.dbo.jwcategory where category_id = a.category_id )as cate_nm ,article_title, article_text   from sjcuwebent.[dbo].[jwarticle] a  where  board_no = '41' and delete_yn = 'N' order by category_id 


C:\Users\lms_test01\AppData\Local\Temp\ipykernel_1636544\267090847.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SjcuWorkingHistory = pd.read_sql(query2, conn)


,cate_nm,article_title,article_text
0,BEST9,"수업 및 시험, 과제 제출은 어떻게 이루어지나요?","<div class=""fr-view""><p>본교의 모든 수업과 시험은 기본적으로 온..."
1,BEST9,신입학으로 입학하기 전에 전 대학에서 취득한 학점을 인정받을 수 있나요?,<div>신입학으로 입학하는 경우에는 다른 대학(2년제/4년제) 또는 학점은행제로 ...
2,BEST9,세종대학교와 어떤 관련이 있나요?,<div>세종대학교와 세종사이버대학교는 동일한 재단 하에 설립된 교육기관입니다.<b...
3,BEST9,졸업 후 대학원에 갈 수 있나요?,"<div class=""fr-view""><div>세종사이버대학교는 고등교육법을 적용받..."
4,BEST9,학교에 직접 가서 출석을 해야 하나요? 지방(해외)거주자인데 괜찮을까요?,<div> 원칙적으로 본교의 수업 및 시험 등 학점이수에 관한 모든 활동은 온라인으...
...,...,...,...
74,시간제 등록,시간제등록생도 장학혜택이 있나요?,"<div class=""fr-view""><div>시간제등록생은 본교에 학적을 두지 않..."
75,시간제 등록,시간제등록생 지원자격은 어떻게 되나요?,<div>고등학교를 졸업한 자 또는 고등학교 졸업학력 검정고시에 합격한 자 및 기타...
76,시간제 등록,시간제등록생 모집기간 및 인원 그리고 선발방법은 어떻게 되나요?,"<div>1) 시간제등록생의 재학기간은 한 학기이므로, 학기마다 등록을 원할 경우 ..."
77,시간제 등록,시간제등록생이 한 학기에 수강신청 가능한 학점은 어떻게 되나요?,"<div><!--StartFragment-->\r\n<p class=""0""><spa..."


In [16]:
from bs4 import BeautifulSoup

for index, row in df_SjcuWorkingHistory.iterrows():

    soup1 = BeautifulSoup(row['article_title'], 'html.parser')
    article_title = soup1.get_text()
    df_SjcuWorkingHistory.at[index, 'article_title'] = article_title


    soup2 = BeautifulSoup(row['article_text'], 'html.parser')
    article_text = soup2.get_text()    
    df_SjcuWorkingHistory.at[index, 'article_text'] = article_text


In [23]:
df_SjcuWorkingHistory

,cate_nm,article_title,article_text
0,BEST9,"수업 및 시험, 과제 제출은 어떻게 이루어지나요?",본교의 모든 수업과 시험은 기본적으로 온라인에서 이루어집니다. 과제는 원칙적으로 강...
1,BEST9,신입학으로 입학하기 전에 전 대학에서 취득한 학점을 인정받을 수 있나요?,신입학으로 입학하는 경우에는 다른 대학(2년제/4년제) 또는 학점은행제로 이미 취득...
2,BEST9,세종대학교와 어떤 관련이 있나요?,세종대학교와 세종사이버대학교는 동일한 재단 하에 설립된 교육기관입니다.\r\n 입학...
3,BEST9,졸업 후 대학원에 갈 수 있나요?,세종사이버대학교는 고등교육법을 적용받는 4년제 온라인 대학교로서 졸업 시 오프라인 ...
4,BEST9,학교에 직접 가서 출석을 해야 하나요? 지방(해외)거주자인데 괜찮을까요?,원칙적으로 본교의 수업 및 시험 등 학점이수에 관한 모든 활동은 온라인으로 진행되...
...,...,...,...
74,시간제 등록,시간제등록생도 장학혜택이 있나요?,"시간제등록생은 본교에 학적을 두지 않고, 정규수업에 시간제로 등록하여 수강하기 때문..."
75,시간제 등록,시간제등록생 지원자격은 어떻게 되나요?,고등학교를 졸업한 자 또는 고등학교 졸업학력 검정고시에 합격한 자 및 기타 법령에 ...
76,시간제 등록,시간제등록생 모집기간 및 인원 그리고 선발방법은 어떻게 되나요?,"1) 시간제등록생의 재학기간은 한 학기이므로, 학기마다 등록을 원할 경우 시간제등록..."
77,시간제 등록,시간제등록생이 한 학기에 수강신청 가능한 학점은 어떻게 되나요?,\n한 학기에 최소 3학점에서 최대 12학점까지 수강신청이 가능합니다. \n(시간제...


In [26]:
filename="./data/sjcu_counsel_menual.csv"
df_SjcuWorkingHistory.to_csv(filename, index=False, encoding='utf-8-sig')